# Linear Regression - 150m

In [49]:
import numpy as np
import pandas as pd
import geopandas as gpd
import shapely
from shapely.wkt import loads
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram
from scipy.cluster.hierarchy import linkage
import matplotlib.pyplot as plt
import seaborn as sns
import altair as alt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
alt.data_transformers.enable("vegafusion")

DataTransformerRegistry.enable('vegafusion')

In [50]:
from IPython.core.display import HTML

# Custom CSS to increase the width of the notebook content
HTML("""
    <style>
        .container {
            width: 90% !important;  /* Set width to 90% of the browser window, adjust as needed */
            max-width: 2000px !important;  /* Optional: limit the max width */
            margin: 0 auto;  /* Center the content */
        }
    </style>
""")

### Choose clear sky day in August

In [51]:
temp =  pd.read_csv('/Users/lisawink/Documents/paper1/data/gap_filled_data_ta_rh.csv')
temp['datetime_UTC']=pd.to_datetime(temp['datetime_UTC'])
temp = temp[temp['variable']=='Ta_deg_C']

In [52]:
# Choose clear sky day (22 August 2023)

temp = temp[(temp['datetime_UTC'].dt.month==8)]
temp = temp[(temp['datetime_UTC'].dt.day==22) | (temp['datetime_UTC'].dt.day==23)]

In [53]:
# Assuming temp is your DataFrame
temp = temp.reset_index()

In [54]:
# Create a multi-selection that chooses the station ID
selection = alt.selection_point(fields=['station_id'], bind='legend', on='click', toggle='event.shiftKey')

# Create the chart
chart = alt.Chart(temp).mark_line().encode(
    x='datetime_UTC:T',
    y='value:Q',
    color=alt.condition(selection, 'station_id:N', alt.value('lightgray'), legend=alt.Legend(columns=2, symbolLimit=0)),
    opacity=alt.condition(selection, alt.value(1), alt.value(0.2))
).add_params(
    selection
).properties(
    title='Temperature over time for each station'
).interactive()

chart.show()

alt.Chart(...)

## 4am & 3pm

In [55]:
#temp = temp[(temp['datetime_UTC'].dt.day==22) & (temp['datetime_UTC'].dt.hour==4)]

In [56]:
#temp = temp[(temp['datetime_UTC'].dt.day==22) & (temp['datetime_UTC'].dt.hour==15)]

In [57]:
temp['datetime_UTC'] = temp['datetime_UTC'].astype(str)

In [58]:
# transpose temp so that each timestep is a column with the station_id as the index
temp = temp.pivot(index='station_id', columns='datetime_UTC', values='value')

## Add precalculated station parameters

In [59]:
# import and drop index
params = gpd.read_parquet('/Users/lisawink/Documents/paper1/data/processed_data/processed_station_params_150_svf.parquet')
params = params[params['station_id']!='FRTECH']
params.index = params['station_id']

In [60]:
items = [
    'BuAre_count', 'BuAre_sum', 'BuAre_mean', 'BuAre_std', 'BuAre_median', 'BuAre_MAD', 'BuAre_IQR', 'BuAre_skew',
    'BuHt_mean', 'BuHt_std', 'BuHt_median', 'BuHt_MAD', 'BuHt_IQR', 'BuHt_skew',
    'BuPer_mean', 'BuPer_std', 'BuPer_median', 'BuPer_MAD', 'BuPer_IQR', 'BuPer_skew',
    'BuLAL_mean', 'BuLAL_std', 'BuLAL_median', 'BuLAL_MAD', 'BuLAL_IQR', 'BuLAL_skew',
    'BuCCD_mean_mean', 'BuCCD_mean_std', 'BuCCD_mean_median', 'BuCCD_mean_MAD', 'BuCCD_mean_IQR', 'BuCCD_mean_skew',
    'BuCor_mean', 'BuCor_std', 'BuCor_median', 'BuCor_MAD', 'BuCor_IQR', 'BuCor_skew',
    'BuCWA_mean', 'BuCWA_std', 'BuCWA_median', 'BuCWA_MAD', 'BuCWA_IQR', 'BuCWA_skew',
    'BuCon_mean', 'BuCon_std', 'BuCon_median', 'BuCon_MAD', 'BuCon_IQR', 'BuCon_skew',
    'BuElo_mean', 'BuElo_std', 'BuElo_median', 'BuElo_MAD', 'BuElo_IQR', 'BuElo_skew',
    'BuERI_mean', 'BuERI_std', 'BuERI_median', 'BuERI_MAD', 'BuERI_IQR', 'BuERI_skew',
    'BuFR_mean', 'BuFR_std', 'BuFR_median', 'BuFR_MAD', 'BuFR_IQR', 'BuFR_skew',
    'BuFF_mean', 'BuFF_std', 'BuFF_median', 'BuFF_MAD', 'BuFF_IQR', 'BuFF_skew',
    'BuFD_mean', 'BuFD_std', 'BuFD_median', 'BuFD_MAD', 'BuFD_IQR', 'BuFD_skew',
    'BuRec_mean', 'BuRec_std', 'BuRec_median', 'BuRec_MAD', 'BuRec_IQR', 'BuRec_skew',
    'BuShI_mean', 'BuShI_std', 'BuShI_median', 'BuShI_MAD', 'BuShI_IQR', 'BuShI_skew',
    'BuSqC_mean', 'BuSqC_std', 'BuSqC_median', 'BuSqC_MAD', 'BuSqC_IQR', 'BuSqC_skew',
    'BuSqu_mean', 'BuSqu_std', 'BuSqu_median', 'BuSqu_MAD', 'BuSqu_IQR', 'BuSqu_skew',
    'BuAdj', 
    'BuIBD', 
    'BuSWR_mean', 'BuSWR_std', 'BuSWR_median', 'BuSWR_MAD', 'BuSWR_IQR', 'BuSWR_skew',
    'BuOri_mean', 'BuOri_std', 'BuOri_median', 'BuOri_MAD', 'BuOri_IQR', 'BuOri_skew',
    'BuAli_mean', 'BuAli_std', 'BuAli_median', 'BuAli_MAD', 'BuAli_IQR', 'BuAli_skew',
    'StrAli_mean', 'StrAli_std', 'StrAli_median', 'StrAli_MAD', 'StrAli_IQR', 'StrAli_skew',
    'StrW_mean', 'StrW_std', 'StrW_median', 'StrW_MAD', 'StrW_IQR', 'StrW_skew',
    'StrWD_mean', 'StrWD_std', 'StrWD_median', 'StrWD_MAD', 'StrWD_IQR', 'StrWD_skew',
    'StrOpe_mean', 'StrOpe_std', 'StrOpe_median', 'StrOpe_MAD', 'StrOpe_IQR', 'StrOpe_skew',
    'StrHW_mean', 'StrHW_std', 'StrHW_median', 'StrHW_MAD', 'StrHW_IQR', 'StrHW_skew',
    'StrLen_mean', 'StrLen_std', 'StrLen_median', 'StrLen_MAD', 'StrLen_IQR', 'StrLen_skew',
    'StrCNS_mean', 'StrCNS_std', 'StrCNS_median', 'StrCNS_MAD', 'StrCNS_IQR', 'StrCNS_skew',
    'BpM_mean', 'BpM_std', 'BpM_median', 'BpM_MAD', 'BpM_IQR', 'BpM_skew',
    'StrLin_mean', 'StrLin_std', 'StrLin_median', 'StrLin_MAD', 'StrLin_IQR', 'StrLin_skew',
    'StrClo400_mean', 'StrClo400_std', 'StrClo400_median', 'StrClo400_MAD', 'StrClo400_IQR', 'StrClo400_skew',
    'StrBet400_mean', 'StrBet400_std', 'StrBet400_median', 'StrBet400_MAD', 'StrBet400_IQR', 'StrBet400_skew',
    'StrSCl_mean', 'StrSCl_std', 'StrSCl_median', 'StrSCl_MAD', 'StrSCl_IQR', 'StrSCl_skew',
    'StrCyc400_mean', 'StrCyc400_std', 'StrCyc400_median', 'StrCyc400_MAD', 'StrCyc400_IQR', 'StrCyc400_skew',
    'StrENR400_mean', 'StrENR400_std', 'StrENR400_median', 'StrENR400_MAD', 'StrENR400_IQR', 'StrENR400_skew',
    'StrGam400_mean', 'StrGam400_std', 'StrGam400_median', 'StrGam400_MAD', 'StrGam400_IQR', 'StrGam400_skew',
    'StrDeg_mean', 'StrDeg_std', 'StrDeg_median', 'StrDeg_MAD', 'StrDeg_IQR', 'StrDeg_skew',
    'StrMes400_mean', 'StrMes400_std', 'StrMes400_median', 'StrMes400_MAD', 'StrMes400_IQR', 'StrMes400_skew',
    'SVF_mean', 'SVF_std', 'SVF_median', 'SVF_IQR'
]

In [61]:
params = params[items]

In [62]:
params_temp = params.merge(temp, left_on=params.index, right_on='station_id',how='left').set_index('station_id')

In [63]:
cooling_rate = params_temp['2023-08-22 17:00:00+00:00'] - params_temp['2023-08-23 05:00:00+00:00']

In [64]:
heating_rate = params_temp['2023-08-22 14:00:00+00:00'] - params_temp['2023-08-22 07:00:00+00:00']

In [65]:
params_temp['cooling_rate'] = cooling_rate
params_temp['heating_rate'] = heating_rate

In [66]:
# standardize data
scaler = StandardScaler()
params_scaled = scaler.fit_transform(params_temp)
params_scaled = pd.DataFrame(params_scaled, columns=params_temp.columns, index=params_temp.index)
params_scaled = params_scaled.dropna(thresh=params_scaled.shape[0] - 7, axis=1)
params_scaled = params_scaled.dropna()

In [67]:
# Define mapping of abbreviations to categories
prefix_to_category = {
    'BuAre': 'Dimension', 'BuHt': 'Dimension', 'BuPer': 'Dimension',
    'BuLAL': 'Dimension', 'BuCCD': 'Dimension', 'BuCor': 'Dimension',
    'CyAre': 'Dimension', 'CyInd': 'Dimension', 'BuCWA': 'Shape',
    'BuCon': 'Shape', 'BuElo': 'Shape', 'BuERI': 'Shape',
    'BuFR': 'Shape', 'BuFF': 'Shape', 'BuFD': 'Shape',
    'BuRec': 'Shape', 'BuShI': 'Shape', 'BuSqC': 'Shape',
    'BuSqu': 'Shape', 'BuAdj': 'Distribution', 'BuIBD': 'Distribution',
    'BuSWR': 'Distribution', 'BuOri': 'Orientation', 'BuAli': 'Orientation',
    'StrAli': 'Orientation', 'StrW': 'Distribution', 'StrWD': 'Distribution',
    'StrOpe': 'Distribution', 'StrHW': 'Distribution', 'StrLen': 'Dimension',
    'StrCNS': 'Dimension', 'BpM': 'Intensity', 'StrLin': 'Shape',
    'StrClo400': 'Connectivity', 'StrBet400': 'Connectivity', 
    'StrSCl': 'Connectivity', 'StrCyc400': 'Connectivity', 
    'StrENR400': 'Connectivity', 'StrGam400': 'Connectivity', 
    'StrDeg': 'Connectivity', 'StrMes400': 'Connectivity',
    'SVF': 'Distribution'
}

unique_prefixes = [item.split('_')[0] for item in items]

# Generate categories data dynamically
categories_data = [
    {'Category': prefix_to_category.get(prefix, 'Unknown'), 'Abbrev.': items[i]}
    for i,prefix in enumerate(unique_prefixes)
]

In [68]:
categories_df = pd.DataFrame(categories_data)

### Set Time

In [69]:
time='cooling_rate'

In [45]:
time = '2023-08-22 04:00:00+00:00'

In [ ]:
linregress_plot('2023-08-22 04:00:00+00:00', 'Nighttime Plot 1')

In [70]:
def linregress_plot(time, axis_title):
    # Step 1: Collect regression results
    results = []
    for param in params_scaled[params_scaled.columns.intersection(items)].columns:
        X = params_scaled[[param]]
        y = params_scaled[time]
        
        # Fit the linear regression model
        model = LinearRegression()
        model.fit(X, y)
        y_pred = model.predict(X)
        
        # Metrics
        gradient = model.coef_[0]
        intercept = model.intercept_
        r_squared = r2_score(y, y_pred)
        
        # Append results
        data = pd.DataFrame({'x': X[param], 'y': y, 'y_pred': y_pred})
        data['param'] = param  # Add parameter name for lookup
        results.append({'param': param, 'gradient': gradient, 'r_squared': r_squared, 'data': data})

    # Create a DataFrame for summary results
    results_df = pd.DataFrame(results).drop(columns=['data'])
    results_df = results_df.merge(categories_df, how='left', left_on='param', right_on='Abbrev.')

    # Combine all data for lookup
    all_data = pd.concat([res['data'] for res in results])

    # Step 2: Altair plots
    # Left plot: R-squared vs Gradient scatter plot
    selection = alt.selection_point(fields=['param'], empty='none', on='click', toggle='event.shiftKey')  # Selection on param
    selection1 = alt.selection_point(fields=['Category'], bind='legend', on='click', toggle='event.shiftKey')
    selection2 = alt.selection_point(fields=['stats'], bind='legend', on='click', toggle='event.shiftKey')

    all_data['station_id'] = all_data.index
    results_df['stats'] = results_df['param'].str.split('_').str[1]

    # Step 2: Add category coloring to scatter plot
    category_colors = alt.Scale(scheme='category10')  # Use a predefined Altair color scheme

    scatter_plot = alt.Chart(results_df).mark_point(size=100).encode(
        x=alt.X('gradient:Q', title='Gradient'),
        y=alt.Y('r_squared:Q', title='R-squared'),
        color=alt.condition(selection1, 'Category:N', alt.value('lightgray')),
        shape=alt.Shape('stats:N', title='Statistic'),
        opacity=alt.condition(selection2, alt.value(1), alt.value(0.2)),
        tooltip=['param', 'Category', 'gradient', 'r_squared']
    ).add_params(
        selection, selection1, selection2
    ).properties(
        title='Gradient vs R-squared',
        width=400,
        height=300
    ).interactive()

    # Step 3: Right plot remains the same
    points = alt.Chart(all_data).transform_filter(
        selection
    ).mark_point().encode(
        x=alt.X('x:Q', title='X'),
        y=alt.Y('y:Q', title=axis_title),
        tooltip=['x', 'y']
    )

    # Create the text labels for the station IDs
    text_labels = alt.Chart(all_data).transform_filter(
        selection
    ).mark_text(
        align='left', 
        baseline='middle', 
        dx=5,  # Slightly offset the text so it doesn't overlap the point
    ).encode(
        x='x',
        y='y',
        text='station_id'  # Use station_id as the label
    )

    line = alt.Chart(all_data).transform_filter(
        selection
    ).mark_line(color='red').encode(
        x='x:Q',
        y='y_pred:Q'
    )

    regression_plot = (points + line + text_labels).properties(
        title='Linear Regression Plot',
        width=400,
        height=300
    )

    # Combine the plots
    final_chart = alt.vconcat(scatter_plot, regression_plot)
    final_chart.show()

### Nighttime Plot

In [71]:
linregress_plot('2023-08-22 04:00:00+00:00', 'Nighttime Plot 1')

alt.VConcatChart(...)

In [72]:
linregress_plot('2023-08-23 04:00:00+00:00', 'Nighttime Plot 2')

alt.VConcatChart(...)

### Daytime Plot

In [73]:
linregress_plot('2023-08-23 15:00:00+00:00', 'Daytime Plot')

alt.VConcatChart(...)

### Heating and Cooling Rate

In [74]:
linregress_plot('cooling_rate', 'Cooling rate')

alt.VConcatChart(...)

In [ ]:
linregress_plot('heating_rate', 'Heating rate')